In [1]:
#following libraries to establish connection between user and ssh remote server and also to access sql in server.
import sshtunnel
import mysql.connector
import pandas as pd
import pymysql
import paramiko
from paramiko import SSHClient
from sshtunnel import SSHTunnelForwarder
from os.path import expanduser
import sys
from pathlib import Path

In [4]:
home = expanduser('~')
# mypkey = paramiko.RSAKey.from_private_key_file(home + pkeyfilepath)
# if you want to use ssh password use - ssh_password='your ssh password', bellow
#below are remote ssh & sql variables. 
sql_hostname = '127.0.0.1'
sql_username = 'ifraz-re'
sql_password = '1q2w3e4r'
sql_main_database = 'ghtorrent_201906'
sql_port = 3306
ssh_host = '163.221.172.90'
ssh_user = 'ifraz-re'
ssh_port = 22
ssh_pwd = 'dangerous123'
# below ssh connection code

with SSHTunnelForwarder(
       (ssh_host, ssh_port),
       ssh_username=ssh_user,
       ssh_password=ssh_pwd,
       remote_bind_address=(sql_hostname, sql_port)) as tunnel:
   
 # below open connection with sql @ server thru ssh. 
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
        passwd=sql_password, db=sql_main_database,
        port=tunnel.local_bind_port)
 # below loading attribute from local csv file.
    file1 = pd.read_csv("gh_login_id_name.csv")
    login_data=file1['login_id']
# below limit the for loop for specific iterations & query to match column between table @ remote server sql and local csv file in a local machine.     

#     limit=100
    j=0   
    return_issues_comm=[]
    for i in login_data:
        sys.stdout.write("\rExtracting data {} / {} --> {}".format(j+1, len(login_data), i))
        sys.stdout.flush()
        temp1=[]
        query = ("Select A.user_id, B.repo_id, A.issue_id, A.comment_id FROM issue_comments A INNER JOIN issues B ON A.issue_id = B.id WHERE A.user_id = '%s'" % i)
        try:
            data = (pd.read_sql_query(query, conn)).values.tolist()
            for n in data:
                temp1.append(n)
            return_issues_comm = return_issues_comm + temp1
        except:
            pass
        
        j+=1
#         if j == limit:
#             break
     
    conn.close()
print("\nFINISH")

Extracting data 12970 / 12970 --> 32440943
FINISH


In [6]:
return_issues_comm

[[41306333, 136434054, 108647522, '494574361'],
 [2495461, 22049695, 79639124, '409707225'],
 [40912624, 107282859, 94748789, '453024465'],
 [40912624, 102511636, 99071094, '465672872'],
 [40912624, 102511636, 99073742, '465682997'],
 [40912624, 102511636, 99075882, '465691157'],
 [40912624, 102511636, 101095663, '471189668'],
 [40912624, 102511636, 101096184, '471191505'],
 [11162886, 5778207, 26039122, '265437639'],
 [11162886, 3112018, 41021137, '267956041'],
 [14946724, 8868310, 93146388, '448703394'],
 [32412739, 74252904, 56415937, '329307304'],
 [32412739, 78797969, 60554779, '345590573'],
 [32412739, 78797969, 60554779, '345590573'],
 [32412739, 78797969, 60548669, '345590638'],
 [32412739, 78797969, 60548669, '345590638'],
 [32412739, 78797969, 62002212, '349089211'],
 [32412739, 87122853, 66723947, '365094929'],
 [37740533, 125765401, 98965920, '466219283'],
 [37740533, 125765401, 98701300, '469808136'],
 [37740533, 125765401, 103393260, '479619301'],
 [7485520, 30995323, 612

In [8]:
mydf= pd.DataFrame(return_issues_comm,columns=['owner_id','project_id','issues_id','issues_comments_id'])
mydf.to_csv("issues_comments_project.csv", index = False)
mydf

,owner_id,project_id,issues_id,issues_comments_id
0,41306333,136434054,108647522,494574361
1,2495461,22049695,79639124,409707225
2,40912624,107282859,94748789,453024465
3,40912624,102511636,99071094,465672872
4,40912624,102511636,99073742,465682997
5,40912624,102511636,99075882,465691157
6,40912624,102511636,101095663,471189668
7,40912624,102511636,101096184,471191505
8,11162886,5778207,26039122,265437639
9,11162886,3112018,41021137,267956041
